In [ ]:
# Manually annotate cells on Napari using a specific guideline
# Saves annotations for each celtype as napari point label .csv (x,y positions)
# authors: Pacome Prompsy
# contact: pacome.prompsy@chuv.ch
# Guenova Lab
# CHUV (Centre Hospitalier Universitaire Vaudois), Lausanne, Suisse


In [ ]:
# this cell is required to run these notebooks on Binder
# if running on Binder, remember to **WAIT 5 SECONDS** after
# running the '%gui qt' cell below. If you get an error,
# click on "Kernel -> Restart" and try again. Make sure also
# that you have a desktop tab open.
import os
#if 'BINDER_SERVICE_HOST' in os.environ:
#    os.environ['DISPLAY'] = ':1.0'

In [ ]:
os.environ["QT_QPA_PLATFORM"] = "wayland"

In [ ]:
##############################################################################################################
##############################################################################################################

# CHANGE HERE SAMPLE :
sample = "ROI-29" # sample number
name = "Annotator" # person annotating

##############################################################################################################
##############################################################################################################

In [ ]:
import os
import pandas as pd
import numpy as np
from tifffile import imread, imwrite
import random
import re

In [ ]:
%gui qt

In [ ]:
import napari
from napari.utils import nbscreenshot

# Create an empty viewer
viewer = napari.Viewer()

In [ ]:
# Change below
output_dir = "/home/localadmin/Documents/Multiplex_Phenotyping_Local/output/manual_annotation_celltype/"
cell_marker_file = "../annotation/cell_markers.csv"
tiff_dir = "../output/input/"
segmentation_dir  = "../output/segmentation/"


In [ ]:
save_dir = os.path.join(output_dir, sample + "-" + name )
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    

In [ ]:
cell_type = pd.read_csv(os.path.join(cell_marker_file))

cell_types_unique = cell_type.cell_type[np.sort(np.unique(cell_type.cell_type, return_index=True)[1])]
cell_types_unique = np.flip(cell_types_unique)

marker_unique = cell_type.marker[np.sort(np.unique(cell_type.marker, return_index=True)[1])]
marker_unique = np.flip(marker_unique)


In [ ]:
background = imread(os.path.join(os.path.join(tiff_dir, sample, "DAPI.tiff")))
viewer.add_image(background, name="background", colormap="gray",contrast_limits=[64000, 65000]);

DAPI = imread(os.path.join(os.path.join(tiff_dir, sample, "DAPI.tiff")))
viewer.add_image(DAPI, name="DAPI", colormap="gray",contrast_limits=[5000, 65000]);


In [ ]:
for i in cell_types_unique:
    df = cell_type[cell_type.cell_type == i]
    chain = i
    for marker in df.marker:
        if marker not in viewer.layers:
            color = cell_type.color_map[cell_type.marker == marker].to_list()[0]
            min_contrast = cell_type.default_min[cell_type.marker == marker].to_list()[0]
            max_contrast = cell_type.default_max[cell_type.marker == marker].to_list()[0]
            visible = cell_type.visible[cell_type.marker == marker].to_list()[0]
            blending = cell_type.blending[cell_type.marker == marker].to_list()[0]
            image = imread(os.path.join(os.path.join(tiff_dir, sample, marker + ".tiff")))
            viewer.add_image(image, name=marker, colormap=color, contrast_limits=[int(min_contrast), int(max_contrast)],
                            blending = blending, visible = visible);


In [ ]:
from skimage.morphology import label
from skimage.segmentation import expand_labels

In [ ]:
cell_boundary = imread(os.path.join(segmentation_dir, sample + "_whole_cell_segmentation_borders.tiff"))
cell_boundary = label(cell_boundary)
cell_boundary = expand_labels(cell_boundary, 2)
cell_boundary[cell_boundary>0] = cell_boundary[cell_boundary>0] + 2000 
viewer.add_image(cell_boundary, name= sample + "-Boundary" , blending= "additive", colormap="red");


In [ ]:
chains = []
for i in cell_types_unique:
    
    df = cell_type[cell_type.cell_type == i]
    chain = i
    for marker in df.marker:
        if np.asarray(df.positive[df.marker == marker])[0] == True:
            chain = chain + str("-") + marker + "+"
        else:
            chain = chain + str("-") + marker + "-"
    print(chain)
    chains = chains + [chain]
    viewer.add_points(name=chain, face_color = df.cell_type_color.to_list()[0],
                      edge_color = "#d4c51bff", edge_width=0.1, opacity=1, size = 30);
    viewer.layers[chain].mode = "add"

In [ ]:
segmentation = imread(os.path.join(os.path.join(segmentation_dir, sample + "_whole_cell_segmentation_borders.tiff")))
viewer.add_labels(segmentation, name="cell_types");

In [ ]:
# Series of shortcut definition to facilitate annotation

In [ ]:
@viewer.bind_key('Shift-S', overwrite = "True")
def save_all(viewer):
    print(save_dir)
    for i in chains:
        viewer.layers[i].save(os.path.join( save_dir, i + '.csv'));

In [ ]:
# Put True if you are reloading
if True:
    chains = []
    for i in cell_types_unique:

        df = cell_type[cell_type.cell_type == i]
        chain = i
        for marker in df.marker:
            if np.asarray(df.positive[df.marker == marker])[0] == True:
                chain = chain + str("-") + marker + "+"
            else:
                chain = chain + str("-") + marker + "-"
        print(chain)
        chains = chains + [chain]
        viewer.layers[chain].face_color = df.cell_type_color.to_list()[0]
        viewer.layers[chain].edge_color = "#d4c51bff"
        viewer.layers[chain].edge_width = 0.1
        viewer.layers[chain].opacity = 1
        viewer.layers[chain].size = 30


In [ ]:
@viewer.bind_key('Shift-Q', overwrite = "True")
def visible_1(viewer):
    point = viewer.layers.selection.active
    celltype_selected = re.sub("-.*", "", point.name)
    df = cell_type[cell_type.cell_type == celltype_selected]
    markers = df["marker"].to_list()
    if len(markers) > 0:
        viewer.layers[markers[0]].visible = not viewer.layers[markers[0]].visible

In [ ]:
@viewer.bind_key('Shift-W', overwrite = "True")
def visible_1(viewer):
    point = viewer.layers.selection.active
    celltype_selected = re.sub("-.*", "", point.name)
    df = cell_type[cell_type.cell_type == celltype_selected]
    markers = df["marker"].to_list()
    if len(markers) > 1:
        viewer.layers[markers[1]].visible = not viewer.layers[markers[1]].visible

In [ ]:
@viewer.bind_key('Shift-E', overwrite = "True")
def visible_1(viewer):
    point = viewer.layers.selection.active
    celltype_selected = re.sub("-.*", "", point.name)
    df = cell_type[cell_type.cell_type == celltype_selected]
    markers = df["marker"].to_list()
    if len(markers) > 2:
        viewer.layers[markers[2]].visible = not viewer.layers[markers[2]].visible

In [ ]:
@viewer.bind_key('Shift-R', overwrite = "True")
def visible_1(viewer):
    point = viewer.layers.selection.active
    celltype_selected = re.sub("-.*", "", point.name)
    df = cell_type[cell_type.cell_type == celltype_selected]
    markers = df["marker"].to_list()
    if len(markers) > 3:
        viewer.layers[markers[3]].visible = not viewer.layers[markers[3]].visible

In [ ]:
@viewer.bind_key('Shift->', overwrite = "True")
def down_contrast_start(viewer):
    point = viewer.layers.selection.active
    celltype_selected = re.sub("-.*", "", point.name)
    df = cell_type[cell_type.cell_type == celltype_selected]
    markers = df["marker"].to_list()
    if len(markers) > 0:
        increment = 50 + round(0.05 * viewer.layers[markers[0]].contrast_limits[0])
        viewer.layers[markers[0]].contrast_limits = [max(0,  viewer.layers[markers[0]].contrast_limits[0] - increment),
                                                     viewer.layers[markers[0]].contrast_limits[1]]

In [ ]:
@viewer.bind_key('Shift-Y', overwrite = "True")
def down_contrast_stop(viewer):
    point = viewer.layers.selection.active
    celltype_selected = re.sub("-.*", "", point.name)
    df = cell_type[cell_type.cell_type == celltype_selected]
    markers = df["marker"].to_list()
    if len(markers) > 0:
        increment = 50 + round(0.05 * viewer.layers[markers[0]].contrast_limits[1])
        viewer.layers[markers[0]].contrast_limits = [viewer.layers[markers[0]].contrast_limits[0],
                                                     max(0, viewer.layers[markers[0]].contrast_limits[1] - increment)]

In [ ]:
@viewer.bind_key('Ctrl-Shift->', overwrite = "True")
def up_contrast_start(viewer):
    point = viewer.layers.selection.active
    celltype_selected = re.sub("-.*", "", point.name)
    df = cell_type[cell_type.cell_type == celltype_selected]
    markers = df["marker"].to_list()
    if len(markers) > 0:
        increment = 50 + round(0.05 * viewer.layers[markers[0]].contrast_limits[0])
        viewer.layers[markers[0]].contrast_limits = [min(65000,  viewer.layers[markers[0]].contrast_limits[0] + increment),
                                                     viewer.layers[markers[0]].contrast_limits[1]]

In [ ]:
@viewer.bind_key('Ctrl-Shift-Y', overwrite = "True")
def up_contrast_end(viewer):
    point = viewer.layers.selection.active
    celltype_selected = re.sub("-.*", "", point.name)
    df = cell_type[cell_type.cell_type == celltype_selected]
    markers = df["marker"].to_list()
    if len(markers) > 0:
        increment = 50 + round(0.05 * viewer.layers[markers[0]].contrast_limits[1])
        viewer.layers[markers[0]].contrast_limits = [viewer.layers[markers[0]].contrast_limits[0],
                                                     min(65000, viewer.layers[markers[0]].contrast_limits[1] + increment)]

In [ ]:
@viewer.bind_key('Shift-D', overwrite = "True")
def visible_1(viewer):
    viewer.layers["DAPI"].visible = not viewer.layers["DAPI"].visible

In [ ]:
@viewer.bind_key('Shift-X', overwrite = "True")
def unvisible_all(viewer):
    for layer in viewer.layers:
        if (not "-" in layer.name) and (layer.name != "background")  and (layer.name != "cell_types"):
            layer.visible = False

In [ ]:
@viewer.bind_key('Shift-C', overwrite = "True")
def visible_all(viewer):
    for layer in viewer.layers:
        if (not "-" in layer.name) and (layer.name != "background")  and (layer.name != "cell_types"):
            layer.visible = True

In [ ]:
@viewer.bind_key('Shift-F', overwrite = "True")
def visible_1(viewer):
    viewer.layers["Cytokeratin"].visible = not viewer.layers["Cytokeratin"].visible